In [1]:
import json

In [2]:
with open('./Corona2.json') as f:
  annotation = json.load(f)

In [3]:
TRAIN_DATA  = []
for e in annotation["examples"]:
    content = e["content"]
    entities = []
    for an in e["annotations"]:
        if len(an["value"]) == len(an["value"].strip()):
            if (len(an['human_annotations']) == 0) or(an['tag_name']!='MedicalCondition'\
             and an['tag_name']!='Medicine'):
                continue
            info = (an["start"],an["end"],an["tag_name"])
            entities.append(info)
            #print(an["start"],an["end"],an["tag_name"])
    if len(entities) > 0:
        TRAIN_DATA.append(([content,{"entities":entities}]))

In [4]:
TRAIN_DATA[0][0],len(TRAIN_DATA)

("While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]",
 26)

In [5]:
import re
def clean_text(text):
  cleaned_text = re.sub(r'\[[0-9]*\]','',text)
  cleaned_text = re.sub(r"'?","", cleaned_text)
  cleaned_text = re.sub(r"'s\b","", cleaned_text)
  cleaned_text = re.sub(r"\(?\)?","", cleaned_text)
  cleaned_text = re.sub(r'[^a-zA-Z]'," ", cleaned_text).strip()
  return cleaned_text

In [6]:
clean_text(TRAIN_DATA[0][0])

'While bismuth compounds Pepto Bismol decreased the number of bowel movements in those with travelers diarrhea  they do not decrease the length of illness  Anti motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease  These agents should be used only if bloody diarrhea is not present   Diosmectite  a natural aluminomagnesium silicate clay  is effective in alleviating symptoms of acute diarrhea in children  and also has some effects in chronic functional diarrhea  radiation induced diarrhea  and chemotherapy induced diarrhea  Another absorbent agent used for the treatment of mild diarrhea is kaopectate   Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults  It has better tolerability than loperamide  as it causes less constipation and flatulence'

In [7]:
cleaned_text = [clean_text(i[0]) for i in TRAIN_DATA]
for i in range(10):
  print(cleaned_text[i])
  print()

While bismuth compounds Pepto Bismol decreased the number of bowel movements in those with travelers diarrhea  they do not decrease the length of illness  Anti motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease  These agents should be used only if bloody diarrhea is not present   Diosmectite  a natural aluminomagnesium silicate clay  is effective in alleviating symptoms of acute diarrhea in children  and also has some effects in chronic functional diarrhea  radiation induced diarrhea  and chemotherapy induced diarrhea  Another absorbent agent used for the treatment of mild diarrhea is kaopectate   Racecadotril an antisecretory medication may be used to treat diarrhea in children and adults  It has better tolerability than loperamide  as it causes less constipation and flatulence

Diarrhea  also spelled diarrhoea  is the condition of having at least three loose  liquid  or watery bowel movements each day  It often lasts fo

In [8]:
sorted(TRAIN_DATA[0][1]['entities'])

[(6, 23, 'Medicine'),
 (25, 37, 'Medicine'),
 (104, 112, 'MedicalCondition'),
 (188, 198, 'Medicine'),
 (360, 371, 'Medicine'),
 (383, 408, 'Medicine'),
 (461, 469, 'MedicalCondition'),
 (535, 543, 'MedicalCondition'),
 (563, 571, 'MedicalCondition'),
 (577, 589, 'Medicine'),
 (679, 689, 'Medicine'),
 (692, 704, 'Medicine'),
 (754, 762, 'MedicalCondition'),
 (823, 833, 'Medicine'),
 (853, 865, 'MedicalCondition'),
 (870, 880, 'MedicalCondition')]

In [9]:
print(cleaned_text[0].split())

['While', 'bismuth', 'compounds', 'Pepto', 'Bismol', 'decreased', 'the', 'number', 'of', 'bowel', 'movements', 'in', 'those', 'with', 'travelers', 'diarrhea', 'they', 'do', 'not', 'decrease', 'the', 'length', 'of', 'illness', 'Anti', 'motility', 'agents', 'like', 'loperamide', 'are', 'also', 'effective', 'at', 'reducing', 'the', 'number', 'of', 'stools', 'but', 'not', 'the', 'duration', 'of', 'disease', 'These', 'agents', 'should', 'be', 'used', 'only', 'if', 'bloody', 'diarrhea', 'is', 'not', 'present', 'Diosmectite', 'a', 'natural', 'aluminomagnesium', 'silicate', 'clay', 'is', 'effective', 'in', 'alleviating', 'symptoms', 'of', 'acute', 'diarrhea', 'in', 'children', 'and', 'also', 'has', 'some', 'effects', 'in', 'chronic', 'functional', 'diarrhea', 'radiation', 'induced', 'diarrhea', 'and', 'chemotherapy', 'induced', 'diarrhea', 'Another', 'absorbent', 'agent', 'used', 'for', 'the', 'treatment', 'of', 'mild', 'diarrhea', 'is', 'kaopectate', 'Racecadotril', 'an', 'antisecretory', 'me

In [10]:
count = 0
def tagging(text, training_ex):
  global count
  tags = ""
  tags = ['O' for _ in range(len(text.split()))]

  text_list = text.split()
  # print(text_list)

  entites = training_ex[1]['entities']
  training_text = training_ex[0]
  for tag in entites:
    temp = clean_text(training_text[tag[0]:tag[1]]).split()

    tag_name = 'med' if (tag[2] == 'Medicine') else 'diag'

    for i in range(len(temp)):
      count += 1
      idx = text_list.index(temp[i])

      tags[idx] = 'B-' + tag_name if (not i) else 'I-' + tag_name
      # print(tags[idx], text_list[idx])
  final_tags = ' '.join([str(elm) for elm in tags])
  final_tags = final_tags.strip().replace(' ', ',')
  # print(final_tags)
  return final_tags

In [11]:
tags = [tagging(cleaned_text[i],TRAIN_DATA[i]) for i in range(len(TRAIN_DATA))]
tags[0]

'O,B-med,I-med,B-med,I-med,O,O,O,O,O,O,O,O,O,O,B-diag,O,O,O,O,O,O,O,O,O,O,O,O,B-med,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-med,O,O,B-med,I-med,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-med,O,O,O,O,O,O,O,O,O,O,O,O,O,B-med,B-med,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-diag,O,B-diag'

In [12]:
import pandas as pd
df = pd.DataFrame({'text': cleaned_text,'tags': tags})
df.shape

(26, 2)

In [13]:
df.to_csv('./data.csv')

In [14]:
df

,text,tags
0,While bismuth compounds Pepto Bismol decreased...,"O,B-med,I-med,B-med,I-med,O,O,O,O,O,O,O,O,O,O,..."
1,Diarrhea also spelled diarrhoea is the condi...,"B-diag,O,O,B-diag,O,O,O,I-diag,O,O,O,O,O,O,O,B..."
2,Antiretroviral therapy ART is recommended for ...,"B-med,I-med,B-med,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,The following drugs are considered as DMARDs ...,"O,O,O,O,O,O,B-med,B-med,B-med,B-med,B-med,B-me..."
4,The goals of treatment are to reduce pain dec...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
5,Bats are the most common source of rabies in h...,"O,O,O,O,O,O,O,B-diag,O,O,O,O,O,O,O,O,O,O,O,O,O..."
6,In following the outbreak of severe acut...,"O,O,O,O,O,B-diag,I-diag,I-diag,I-diag,B-diag,O..."
7,Bacterial vaginosis is caused by bacteria that...,"O,O,O,O,O,O,O,O,I-diag,B-diag,O,O,O,O,O,O,O,O,..."
8,One of the bacterial diseases with the highest...,"O,O,O,O,O,O,O,O,O,O,O,B-diag,O,O,O,O,O,O,O,O,O..."
9,Although the vast majority of bacteria are har...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [15]:
count

286